<a href="https://colab.research.google.com/github/opalclouds/DataCo-global-policies-RAG-pipeline/blob/main/DataCo_policies_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:

!pip install kaggle


from google.colab import files
files.upload()


!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d sghhim/dataco-global-policy-dataset


!unzip dataco-global-policy-dataset.zip


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/sghhim/dataco-global-policy-dataset
License(s): MIT
dataco-global-policy-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  dataco-global-policy-dataset.zip
replace Anti-Counterfeit and Product Authenticity Policy.pdf? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: Anti-Counterfeit and Product Authenticity Policy.pdf  
  inflating: COC.pdf                 
  inflating: Circular Economy.pdf    
  inflating: Communication and Crisis Management Policy for DataCo Global.pdf  
  inflating: Continuous Improvement.pdf  
  inflating: Cost Reduction.pdf      
  inflating: Data Security.pdf       
  inflating: DataCo Global Capacity Planning Policy.pdf  
  inflating: DataCo Global Contract Management and Negotiation Policy.pdf  
  inflating: DataCo Global Warehouse and Storage Policy.pdf  
  inflating: Dataco Global Change Management Policy for Supply Chain P

In [50]:
!pip install pdfplumber

import os
import pdfplumber

folder = "/content/"
all_texts = []


for file in os.listdir(folder):
    if file.endswith(".pdf"):
        path = os.path.join(folder, file)
        with pdfplumber.open(path) as pdf:
            text = ""
            for page in pdf.pages:
                text += page.extract_text() + "\n"
            all_texts.append({"file": file, "text": text})

print(all_texts[0]["file"])
print(all_texts[0]["text"][:500])

Dataco Global Transportation and Logistics Policy.pdf
Dataco Global Transportation and Logistics
Policy
1. Introduction
This comprehensive policy establishes the framework for all transportation and logistics operations
at Dataco Global. It outlines our commitment to efficient, sustainable, and compliant logistics
practices while setting quantifiable targets and measurable standards for all aspects of our supply
chain.
2. Scope and Objectives
Scope: This policy applies to all employees, contractors, and third-party service providers involved
in tra


In [51]:
all_texts[0]["file"]
all_texts[0]["text"]

'Dataco Global Transportation and Logistics\nPolicy\n1. Introduction\nThis comprehensive policy establishes the framework for all transportation and logistics operations\nat Dataco Global. It outlines our commitment to efficient, sustainable, and compliant logistics\npractices while setting quantifiable targets and measurable standards for all aspects of our supply\nchain.\n2. Scope and Objectives\nScope: This policy applies to all employees, contractors, and third-party service providers involved\nin transportation and logistics activities for Dataco Global.\nObjectives:\nReduce transportation costs by 15% within 12 months\nAchieve 98.5% on-time delivery performance\nDecrease carbon emissions from logistics operations by 20% by 2027\nMaintain a safety incident rate below 0.5 per 100,000 miles traveled\nEnsure 100% compliance with all relevant transportation regulations\n3. Fleet Management\nVehicle Acquisition and Maintenance:\nAll fleet vehicles must undergo maintenance inspections e

In [52]:
len(all_texts)

30

In [56]:
def chunk_text(text, chunk_size=800, overlap=100):
  chunks=[]
  start=0
  while start<len(text):
    chunk=text[start:start+chunk_size]
    chunks.append(chunk)
    start=start+chunk_size-overlap
  return chunks

In [57]:
chunked_doc=[]
for doc in all_texts:
  chunks=chunk_text(doc["text"])
  for I in chunks:
    chunked_doc.append({"file":doc["file"],"text":I})




In [58]:
!pip install transformers

In [59]:
!pip install sentence_transformers

In [60]:
from sentence_transformers import SentenceTransformer
model=SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [61]:
texts=[doc["text"] for doc in chunked_doc]

In [62]:
files=[doc["file"] for doc in chunked_doc]

In [63]:
embeddings = model.encode(texts, batch_size=32, show_progress_bar=True)

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

In [64]:
import pandas as pd

In [65]:
len(chunked_doc)

198

In [66]:
df=pd.DataFrame({
    "text": texts,
    "file": files,
    "embedding": embeddings.tolist()
})

In [67]:
df.to_parquet("embeddings.parquet")

In [68]:
df.shape

(198, 3)

In [69]:
df.iloc[:5]

,text,file,embedding
0,Dataco Global Transportation and Logistics\nPo...,Dataco Global Transportation and Logistics Pol...,"[-0.026870813220739365, -0.022786715999245644,..."
1,ics operations by 20% by 2027\nMaintain a safe...,Dataco Global Transportation and Logistics Pol...,"[-0.01952327974140644, -0.042716097086668015, ..."
2,90%\nMaximum continuous driving time: 4 hours...,Dataco Global Transportation and Logistics Pol...,"[0.05620076507329941, 0.030185243114829063, -0..."
3,te optimization for all deliveries to reduce m...,Dataco Global Transportation and Logistics Pol...,"[-0.012304450385272503, 0.013703043572604656, ..."
4,f 98% or higher\n6. Transportation Technology\...,Dataco Global Transportation and Logistics Pol...,"[0.008518839254975319, 0.049753714352846146, 0..."


In [70]:
!pip install faiss-cpu

In [71]:
import faiss

In [72]:
d=embeddings[1]

In [74]:
index=faiss.IndexFlatL2(len(d))
# L2 meaning eucleadian distances

In [76]:
import numpy as np

In [77]:
#creating a index and then adding our embeddings to index
index.add(np.array(embeddings,dtype="float32"))

In [78]:
faiss.write_index(index,"embedding.index")

In [80]:
def search_query(query, k):

  q_emb=model.encode([query])[0].astype("float32")

  distance,idx=index.search(q_emb.reshape(1,-1),k)

  results=[]
  for i in idx[0]:
    results.append({
        "file":files[i],
        "text":texts[i],
        "index":i
    })

    return results

In [81]:
user_query=input("enter question")

enter questionenvironment goals?


In [82]:
results=search_query(user_query,k=5)
for r in results:
  print(r["file"])
  print(r["text"])


Environmental Sustainability.pdf
ctions.
5. Water Conservation
All offices to install water-saving fixtures, targeting a minimum 20% reduction in water
consumption (compared to 2024) by 2027.
Require facility managers to submit quarterly water usage reports and corrective action plans if
targets are missed.
6. Employee Engagement and Training
All employees must complete mandatory environmental sustainability training by the end of
2025, refreshed annually.
Establish a “Green Champions” program with at least one representative per department to
oversee sustainability initiatives and report monthly progress.
7. Performance Measurement and Reporting
Dataco Global will publish an annual Environmental Impact Report, audited by an external
party, disclosing progress toward each quantifiable goal stated in this policy.
Key Perfo
